In [1]:
#import sampleCleaningTransform
from fmri_sample_cleaning_transform import get_affected_samples
from fmri_cleaner import remove_samples
import os
import numpy as np
import mvpa2
import nibabel
import pickle
import gzip
from mvpa2.suite import *
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
%matplotlib inline
#from sampleCleaningTransform import printSomething

ImportError: No module named 'fmri_sample_cleaning_transform'

In [ ]:
# get the location of the files
trajectory_path = "..\\data\\story\\smoothedTrajectoriesDifferentWindowSizes\\smoothTrajRawValenceMean_win10.pkl"
subject_timeline_path = "..\\data\\plosone\\subject_3.mat"
bold_fname = os.path.join('..', 'data', 'qub', '4Dw.nii')
mask_fname = os.path.join('..', 'data', 'qub', 'struct', 'final_mask_w.hdr')

In [ ]:
ds_removal_transform = get_affected_samples(trajectory_path, subject_timeline_path)

In [2]:
ds = remove_samples(bold_fname, mask_fname, ds_removal_transform)

NameError: name 'remove_samples' is not defined

In [3]:
len(ds)

NameError: name 'ds' is not defined

In [ ]:
#get the trajectory
with open('..\\data\\story\\smoothedTrajectoriesDifferentWindowSizes\\smoothTrajRawValenceMean_win10.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    original_trajectory = u.load()

In [ ]:
plt.plot(original_trajectory)
plt.title('original_trajectory')
plt.show()

In [ ]:
#downsample the trajectory to fit match the number of fMRI sample
resampled_trajectory = resample(original_trajectory, len(ds.samples))

In [ ]:
ds.sa['targets'] = resampled_trajectory
#detrender = PolyDetrendMapper(polyord=1)
#ds = ds.get_mapped(detrender)
#training_set = ds[400 : ]
#test_set = ds[ : 400]

In [ ]:
plt.plot(ds.sa.targets)
plt.title('ds')
plt.show()

In [ ]:
set_type = []
test_start_pos = 800
test_end_pos = 900
for i in range(0, len(ds)):
    if i >= test_start_pos and i <= test_end_pos:
        set_type.append("test")
    else:
        set_type.append("train")
        
ds.sa['set_type'] = set_type

In [ ]:
training_set = ds[ds.sa.set_type == 'train']
test_set = ds[ds.sa.set_type == 'test']

In [ ]:
plt.plot(training_set.sa.targets)
plt.title('training_set')
plt.show()

In [ ]:
plt.plot(test_set.sa.targets)
plt.title('test_set')
plt.show()

In [ ]:
clf_1 = SKLLearnerAdapter(DecisionTreeRegressor())

In [ ]:
clf_1.train(training_set)

In [ ]:
predictions = clf_1.predict(test_set)

In [ ]:
plt.plot(test_set.sa.targets)
plt.ylabel('test_set Response')
plt.show()

In [ ]:
plt.plot(predictions)
plt.ylabel('predictions Response')
plt.show()

In [ ]:
len(resampled_trajectory)

In [ ]:
plt.plot(ds.sa.targets)
plt.ylabel('ds Response')
plt.show()

In [ ]:
combined_results = np.concatenate((training_set.sa.targets, predictions), axis=0)
plt.plot(combined_results)
plt.title('combined_results')
plt.show()

In [ ]:
clf_2 = kNN(k=3, dfx=one_minus_correlation, voting='majority')

In [ ]:
clf_2.train(training_set)

In [ ]:
predictions2 = clf_2.predict(test_set.samples)

In [ ]:
plt.plot(test_set.sa.targets)
plt.ylabel('test_set targets')
plt.show()

In [ ]:
plt.plot(predictions2)
plt.ylabel('predictions')
plt.show()